In [ ]:
# Generate attribution maps for GradCAM and EP to run pointing game and localization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torch.nn.functional as F
import torchvision
from torchvision import models
from torchvision import transforms

import torch
import torchvision

torch.set_num_threads(1)
torch.manual_seed(0)
np.random.seed(0)

from torchvision.models import *
from visualisation.core.utils import device, image_net_postprocessing
from torch import nn
from operator import itemgetter
from visualisation.core.utils import imshow
from IPython.core.debugger import Tracer


In [ ]:
layer = 4
    
model = resnet34(pretrained=True).to(device)
model.eval()

In [ ]:
# %matplotlib notebook 
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch 
from utils import *
from PIL import Image

plt.rcParams["figure.figsize"]= 16,8


def make_dir(path):
    if os.path.isdir(path) == False:
        os.mkdir(path)

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imshow

from visualisation.core.utils import device 
from PIL import Image
from torchvision.transforms import ToTensor, Resize, Compose, ToPILImage
from visualisation.core import *
from visualisation.core.utils import image_net_preprocessing

size = 224

# Pre-process the image and convert into a tensor
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
])

img_num = 50

# methods = ['Conf', 'GradCAM', 'EP', 'SHAP', 'NNs', 'PoolNet', 'AIonly']
methods = ['Conf', 'GradCAM', 'EP', 'NNs', 'PoolNet', 'AIonly']

task = 'Adversarial_Nat'
# Adversarial_Nat

output_path = '/home/dexter/Downloads/Human_experiments/Visualization'
dataset_path = '/home/dexter/Downloads/Human_experiments/Dataset'
make_dir('{}/{}'.format(output_path,task))
make_dir('{}/{}/Conf/'.format(output_path,task))
make_dir('{}/{}/GradCAM/'.format(output_path,task))
make_dir('{}/{}/EP/'.format(output_path,task))
# make_dir('Pilot_study2/{}/SHAP/'.format(task))
make_dir('{}/{}/PoolNet/'.format(output_path,task))
make_dir('{}/{}/NNs/'.format(output_path,task))
# test_image_paths = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/mixed_images/*.*'.format(task))
corrects_bin1 = glob.glob('{}/{}/correct_bin1_images/*.*'.format(dataset_path,task))
wrongs_bin1 = glob.glob('{}/{}/wrong_bin1_images/*.*'.format(dataset_path,task))
corrects_bin2 = glob.glob('{}/{}/correct_bin2_images/*.*'.format(dataset_path,task))
wrongs_bin2 = glob.glob('{}/{}/wrong_bin2_images/*.*'.format(dataset_path,task))
corrects_bin3 = glob.glob('{}/{}/correct_bin3_images/*.*'.format(dataset_path,task))
wrongs_bin3 = glob.glob('{}/{}/wrong_bin3_images/*.*'.format(dataset_path,task))
# print(test_image_paths)
# test_inputs = list()
test_images = list()

modified_img = list()

In [ ]:
# Added for loading ImageNet classes
def load_imagenet_label_map():
    """
    Load ImageNet label dictionary.
    return:
    """

    input_f = open("input_txt_files/imagenet_classes.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(": ")
        (num, label) = (int(parts[0]), parts[1].replace('"', ""))
        label_map[num] = label

    input_f.close()
    return label_map


# Added for loading ImageNet classes
def load_imagenet_id_map():
    """
    Load ImageNet ID dictionary.
    return;
    """

    input_f = open("input_txt_files/synset_words.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(" ")
        (num, label) = (parts[0], ' '.join(parts[1:]))
        label_map[num] = label

    input_f.close()
    return label_map

def convert_imagenet_label_to_id(label_map, key_list, val_list, prediction_class):
    """
    Convert imagenet label to ID: for example - 245 -> "French bulldog" -> n02108915
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    class_to_label = label_map[prediction_class]
    prediction_id = key_list[val_list.index(class_to_label)]
    return prediction_id


# gt_dict = load_imagenet_validation_gt()
id_map = load_imagenet_id_map()
label_map = load_imagenet_label_map()

key_list = list(id_map.keys())
val_list = list(id_map.values())

def convert_imagenet_id_to_label(label_map, key_list, val_list, class_id):
    """
    Convert imagenet label to ID: for example - n02108915 -> "French bulldog" -> 245
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    return key_list.index(str(class_id))

from torchray.attribution.extremal_perturbation import extremal_perturbation, contrastive_reward
from torchray.attribution.grad_cam import grad_cam
import PIL.Image


def get_EP_saliency_maps(model, path):
        img_index = (path.split('.jpeg')[0]).split('images/')[1]
        img = PIL.Image.open(path)
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        category_id_1 = index[0][0].item()

        masks, energy = extremal_perturbation(
            model, x, category_id_1,
            areas=[0.025, 0.05, 0.1, 0.2],
            num_levels=8,
            step=7,
            sigma=7 * 3,
            max_iter=800,
            debug=False,
            jitter=True,
            smooth=0.09,
            perturbation='blur'
        )
        saliency = masks.sum(dim=0, keepdim=True)
        saliency = saliency.detach()
        saliency_t = saliency.cpu().detach().numpy()[0, 0, :]
        
        saliency_path = 'saliency_maps/EP_resnet34/'
        if not (os.path.exists(saliency_path)):
            os.mkdir(saliency_path)
        img_index = img_index[10:33]
        np.save(os.path.join(saliency_path, "{}.npy".format(img_index)), saliency_t)

        return (saliency[0].to('cpu')), (masks[0].to('cpu')), (masks[1].to('cpu')), (masks[2].to('cpu')) , (masks[3].to('cpu'))

In [ ]:
# import os
import os.path
from visualisation.core.utils import tensor2cam
postprocessing_t = image_net_postprocessing
import cv2 as cv
import sys

imagenet_train_path = '/home/dexter/Downloads/train'

## Creating colormap
cMap = 'Reds'

id_list= list()
conf_dict = {}
eps=1e-5
cnt = 0
K = 3 # Change to your expected number of nearest neighbors

In [ ]:
method = 'Conf'
test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

In [ ]:
method = 'GradCAM'

for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
    
    input_image = img.resize((size,size), Image.ANTIALIAS)

    image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
    print(image_name)

    # Get the ground truth of the input image
    gt_label_id = image_path.split('val_')[1][9:18]

    
    gt_label = id_map.get(gt_label_id)
    id = key_list.index(gt_label_id)
    gt_label = gt_label.split(',')[0]


    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]

    conf_dict[image_name] = predicted_confidence

#     GRAD-CAM
    saliency = grad_cam(
        model, x, input_category_id,
        saliency_layer='layer4',
        resize=True
    )
#     Tracer()()
    img_index = (image_path.split('.jpeg')[0]).split('images/')[1]
    img_index = img_index[10:33]
    if not (os.path.exists(saliency_path)):
        os.mkdir(saliency_path)
    saliency_path = 'saliency_maps/GradCAM_resnet34/'
    saliency *= 1.0/saliency.max()
    GradCAM = saliency[0][0].cpu().detach().numpy()
    np.save(os.path.join(saliency_path, "{}.npy".format(img_index)), GradCAM)

In [ ]:
method = 'EP'
for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    # Extremal Perturbation
    get_EP_saliency_maps(model, image_path)

In [ ]:
method = 'PoolNet'
from shutil import copyfile, rmtree
def rm_and_mkdir(path):
    if os.path.isdir(path) == True:
        rmtree(path)
    os.mkdir(path)

# Prepare dataset
rm_and_mkdir('/home/dexter/Downloads/run-0/run-0-sal-p/')
rm_and_mkdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/')
if os.path.isdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'):
    os.remove('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst')

src_paths = test_image_paths
for src_path in src_paths:
    dst_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/' + src_path.split('images/')[1]
    copyfile(src_path, dst_path)

cmd = 'ls /home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/ > /home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'
os.system(cmd)
cmd = 'python /home/dexter/Downloads/PoolNet-master/main.py --mode=\'test\' --model=\'/home/dexter/Downloads/run-0/run-0/models/final.pth\' --test_fold=\'/home/dexter/Downloads/run-0/run-0-sal-p/\' --sal_mode=\'p\''
os.system(cmd)


npy_file_paths = glob.glob('/home/dexter/Downloads/run-0/run-0-sal-p/*.*')
print(len(npy_file_paths))
for idx, npy_file_path in enumerate(npy_file_paths): 
    img_index = npy_file_path.split('/home/dexter/Downloads/run-0/run-0-sal-p/')[1]
    img_index = img_index[10:33]
    saliency_path = 'saliency_maps/SOD_resnet34/'
    dst_path = saliency_path + img_index + '.npy'
    copyfile(npy_file_path, dst_path)